# Extract Buildings from Mask and Image

In [10]:
import os
from tqdm import tqdm
from PIL import Image
import numpy as np

OUTPUT_PATH = 'output_mask2/'
MASK_PATH = 'train/targets/'
IMG_PATH = 'train/images/'
CROP_SIZE = (64, 64)

if not os.path.isdir(OUTPUT_PATH):
    os.mkdir(OUTPUT_PATH)

imgs_path = os.listdir(IMG_PATH)
imgs_path = [x for x in imgs_path if ('post') in x]
masks_path = os.listdir(MASK_PATH)
masks_path = [x for x in masks_path if ('post') in x]

print(len(imgs_path))

2799


# Original Size

In [ ]:
from skimage.measure import label, regionprops

prog_bar = tqdm(tuple(zip(masks_path, imgs_path)))

output_idx = 0
for mask_path, img_path in prog_bar:
    mask = np.array(Image.open(MASK_PATH+mask_path))
    img = np.array(Image.open(IMG_PATH+img_path))
    
    mask_label = label(mask)
    regions = regionprops(mask_label)

    for prop in regions:
        # isolate region
        bdr = prop.bbox
        crop_mask = mask_label.copy()
        crop_mask = (crop_mask == prop.label).astype(np.uint8)

        # crop image and mask
        cropped_img = crop_mask[:,:,np.newaxis] * img
        cropped_img = cropped_img[bdr[0]:bdr[2], bdr[1]:bdr[3]]
        cropped_mask = crop_mask * mask # cropped to get label
        cropped_mask = cropped_mask[bdr[0]:bdr[2], bdr[1]:bdr[3]]

        # image label
        cls = np.max(cropped_mask).item() - 1 # -1 for 0 indexing
        if not os.path.isdir(OUTPUT_PATH + str(cls)):
            os.mkdir(OUTPUT_PATH + str(cls))
        
        # resize and save to output
        save_path = f"{OUTPUT_PATH}{cls}/{output_idx}.png"
        output_idx += 1
        Image.fromarray(cropped_img).resize(CROP_SIZE, Image.NEAREST).save(save_path)

print(output_idx)

# Tiled

In [ ]:
def tile_image(image, size):
    tiles = []
    for x in range(image.shape[0] // size[0]):
        for y in range(image.shape[1] // size[1]):
            top = size[0] * x
            left = size[1] * y

            bottom = top + size[0]
            right = left + size[1]
            
            #print(f"{top_left}, {bottom_right}")
            tiles.append(image[top:bottom, left:right])
    return tiles


TILE_SIZE = (256, 256)
prog_bar = tqdm(tuple(zip(masks_path, imgs_path)))

output_idx = 0
for mask_path, img_path in prog_bar:
    img0 = np.array(Image.open(IMG_PATH+img_path))
    mask0 = np.array(Image.open(MASK_PATH+mask_path))

    tiled_img = tile_image(img0, TILE_SIZE)
    tiled_mask = tile_image(mask0, TILE_SIZE)

    for img,mask in tuple(zip(tiled_img, tiled_mask)):
        mask_label = label(mask)
        regions = regionprops(mask_label)

        for prop in regions:
            # isolate region
            bdr = prop.bbox
            crop_mask = mask_label.copy()
            crop_mask = (crop_mask == prop.label).astype(np.uint8)

            # crop image and mask
            cropped_img = crop_mask[:,:,np.newaxis] * img
            cropped_img = cropped_img[bdr[0]:bdr[2], bdr[1]:bdr[3]]
            cropped_mask = crop_mask * mask # cropped to get label
            cropped_mask = cropped_mask[bdr[0]:bdr[2], bdr[1]:bdr[3]]

            # image label
            cls = np.max(cropped_mask).item() - 1 # -1 for 0 indexing
            if not os.path.isdir(OUTPUT_PATH + str(cls)):
                os.mkdir(OUTPUT_PATH + str(cls))
            
            # resize and save to output
            save_path = f"{OUTPUT_PATH}{cls}/{output_idx}.png"
            output_idx += 1
            Image.fromarray(cropped_img).resize(CROP_SIZE, Image.NEAREST).save(save_path)

print(output_idx)